In [1]:
import pathlib
from Utils.wetland_selection    import get_top_scenes
from Utils.treatment_regression import logistic_process
from Utils.outcome_syndata_generator import generate_synthetic_outcome
from Utils.treatment_syndata_generator import generate_synthetic_treatment
from Utils.outcome_regression import regression_process
from Utils.convolution import make_exp_kernel
from Utils.outcome_post import generate_post
import numpy as np

In [2]:
# top scenes by wetland development
top_df = get_top_scenes(num_scenes=3566)
scene_ids = top_df['scene'].astype(str).tolist()

# define folder paths for rasters
DATA_DIR = pathlib.Path('/Users/sayedmorteza/Library/CloudStorage/Box-Box/Hetwet_Data')         

# folders
folders = {
    'wet': DATA_DIR / 'WETLAND_DEV_1996_2016',
    'dem': DATA_DIR / 'DEM',
    'cap': DATA_DIR / 'CAPITAL_1996',
    'claims_96': DATA_DIR / 'LOG_CLAIMS_1996',
    'claims_16': DATA_DIR / 'LOG_CLAIMS_2016',

}

In [3]:
# estimate treatment logistic regression parameters
theta = 0.5  # classification threshold
logit_res = logistic_process(
    scene_ids=scene_ids,
    folders=folders,
    threshold=theta,
    regularization_C=1.0,
    noise_type='gaussian',
    noise_sd=0.005,
    verbose=False
)

In [4]:
# generate synthetic treatment
generate_synthetic_treatment(
    scene_ids=scene_ids,
    folders=folders,
    logit_pipe=logit_res,
    threshold=theta,
    noise_sd=0.005,
    noise_type='gaussian'
)


In [3]:
# estimate outcome linear regression parameters
outcome_reg = regression_process(
    scene_ids=scene_ids,
    folders=folders,
    noise_sd=0.1,
    noise_type='gaussian',
    verbose=False
)

In [4]:
# generate synthetic baseline outcomes:
generate_synthetic_outcome(
    scene_ids = scene_ids,
    folders = folders,
    reg_pipe = outcome_reg,
    target_shape = (8, 8),
    noise_type = "gaussian",
    noise_sd = 0.1,
    results_dir = None
)

In [5]:
# build your spillover kernel
CELL_SIZE = 30 # cell size in meters
LAMBDA    = 1000.0
TRUNCATE = 1
KERNEL    = make_exp_kernel(lam_m=LAMBDA, cell_size_m=CELL_SIZE, truncate=TRUNCATE)
print(f"Number of Affected Nearby Squares: {KERNEL.shape}")

Number of Affected Nearby Squares: (67, 67)


In [ ]:
import os
import random
import logging
from pathlib import Path
import numpy as np
from PIL import Image
from Utils.load_and_resize import load_and_resize
from Utils.convolution import upstream_only_KuT, downstream_only_KuT
import matplotlib.pyplot as plt

Bbase = 0.02
Beta1 = 0.1
Beta2 = 1

base = (
            '~/Library/CloudStorage/Box-Box/Hetwet_Data/Synthetic'
        )

results_dir = Path(os.path.expanduser(base))

treatment_dir = results_dir / 'Treatment'
outcome_dir   = results_dir / 'Outcome'

sid = 1767

paths = {
            'dem':     Path(folders['dem'])      / f"DEM_{sid}.tiff",
            'cap':     Path(folders['cap'])      / f"CAPITAL_1996_{sid}.tiff",
            'treat':   treatment_dir            / f"treatment_scene_{sid}_gaussian.tiff",
            'actual':  outcome_dir              / f"outcome_scene_{sid}_gaussian.tiff"
        }

print(paths['treat'])                 # full path
print(paths['treat'].exists())       # check if it exists

# changing the
target_shape = (256, 256)
# Load & resize to WETLAND size
dem_r    = load_and_resize(paths['dem'],    target_shape, Image.BILINEAR)
cap_r    = load_and_resize(paths['cap'],    target_shape, Image.BILINEAR)
treat_r  = load_and_resize(paths['treat'],  target_shape, Image.NEAREST)

coarse_shape = (8, 8)

# Load & resize to Claims size
actual_r = load_and_resize(paths['actual'], coarse_shape, Image.NEAREST)

# Cast
dem    = dem_r.astype(np.float32)
cap    = cap_r.astype(np.float32)
treat = treat_r.astype(np.float32)
actual = actual_r.astype(np.float32)

# Compute θ(x)
dem_safe = np.maximum(dem, 1.0)
theta = Bbase * (1.0
                    + Beta1 * np.log1p(cap)
                    + Beta2 / dem_safe)

# Compute upstream spillover on coarse grid
KuT_up = upstream_only_KuT(treat, dem, KERNEL)


/Users/sayedmorteza/Library/CloudStorage/Box-Box/Hetwet_Data/Synthetic/Treatment/outcome_scene_1767_gaussian.tiff
False


AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
generate_post(
    scene_ids=scene_ids,
    folders=folders,
    KERNEL=KERNEL,
    Bbase = 0.02,
    Beta1 = 0.1,
    Beta2 = 1
)